# RSMACD 指標選股

RSMACD 指標（MACD 強弱指標）結合 MACD 的穩定性與 RSI 的超買超賣判斷能力，
將 RSI 的算法套用在 MACD 線上，產生 0-100 範圍的標準化指標值。

## 6 種買賣信號

### 買入信號
- **綠色箭頭**：RSMACD 曲線轉折向上，提示加碼或進場
- **綠色三角形**：RSMACD 由超賣區（30以下）突破 30，提示趨勢反轉（**主要選股信號**）
- **綠色小球**：RSMACD 在低位轉折向上並突破 50，提示趨勢反轉或加碼

### 賣出信號
- **紅色箭頭**：RSMACD 曲線轉折向下，提示減碼或獲利了結
- **紅色倒三角形**：RSMACD 由超買區（70以上）跌破 70，提示趨勢反轉
- **紅色小球**：RSMACD 在高位轉折向下並跌破 50，提示獲利了結

## 確認法操作邏輯
1. 鎖定關鍵 K 線：出現信號的那根 K 線
2. 進場確認：後續 K 線收盤價突破關鍵 K 線高點 → 確認進場
3. 失敗處理：後續收盤價跌破關鍵 K 線低點 → 觀望
4. 停損設定：關鍵 K 線的低點

## 篩選條件
- 最新交易日出現「綠色三角形」信號
- 5日平均成交量 > 1000張

In [ ]:
import os
import sys
from datetime import date
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
from lightweight_charts import JupyterChart

In [ ]:
from finlab import data
import finlab

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.google_api import gspread_manager as gsm
from proj_util_pkg.common import tw_stock_topic as tst

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [ ]:
# 資訊輸出Google SpreadSheet 表單參數設定
GSPERAD_SHEET_KEY = os.environ.get('gspread_wb_key')
OUTPUT_GSHEET_NAME = '選股清單07'

In [ ]:
# 本地報表輸出路徑
REPORT_PATH = os.environ.get('report_path')

## 外部資料讀取

In [ ]:
# 讀取台股資訊
close = data.get("price:收盤價", save_to_storage=True)
open_price = data.get("price:開盤價", save_to_storage=True)
high = data.get("price:最高價", save_to_storage=True)
low = data.get("price:最低價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
stock_info = data.get('company_basic_info', save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)

## RSMACD 指標計算函數定義

In [ ]:
def calculate_rsmacd(fast=12, slow=26, signal=9, rsi_period=14):
    """
    計算 RSMACD 指標（MACD 強弱指標）
    
    原理：將 RSI 的算法套用在 MACD 線上，將 MACD 正規化到 0-100 範圍
    
    Parameters
    ----------
    fast : int
        MACD 快線週期，預設 12
    slow : int
        MACD 慢線週期，預設 26
    signal : int
        MACD 信號線週期，預設 9
    rsi_period : int
        RSI 計算週期，預設 14
    
    Returns
    -------
    tuple of pd.DataFrame
        (rsmacd, macd_line, macd_signal, macd_hist)
    """
    # Step 1: 透過 FinLab API 計算 MACD
    macd_line, macd_signal, macd_hist = data.indicator(
        'MACD',
        adjust_price=False,
        resample='D',
        fastperiod=fast,
        slowperiod=slow,
        signalperiod=signal,
        save_to_storage=True,
    )
    
    # Step 2: 計算 MACD 線的日變化量
    macd_delta = macd_line.diff()
    
    # Step 3: 分離漲跌
    gain = macd_delta.clip(lower=0)       # 正值保留，負值歸零
    loss = (-macd_delta).clip(lower=0)    # 負值取絕對值，正值歸零
    
    # Step 4: 使用 Wilder's smoothing（等同 TA-Lib RSI 計算方式）
    avg_gain = gain.ewm(alpha=1/rsi_period, min_periods=rsi_period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/rsi_period, min_periods=rsi_period, adjust=False).mean()
    
    # Step 5: 計算 RS 和 RSMACD
    rs = avg_gain / avg_loss.replace(0, np.nan)  # 避免除以零
    rsmacd = 100 - (100 / (1 + rs))
    
    # 處理邊界情況
    rsmacd = rsmacd.where(avg_loss != 0, 100.0)  # avg_loss 為零 → 全漲 → 100
    rsmacd = rsmacd.where(avg_gain != 0, 0.0)    # avg_gain 為零 → 全跌 → 0
    
    return rsmacd, macd_line, macd_signal, macd_hist


# 計算 RSMACD
rsmacd, macd_line, macd_signal, macd_hist = calculate_rsmacd()

print("RSMACD 計算完成")
print(f"RSMACD 形狀: {rsmacd.shape}")
print(f"RSMACD 值域: {rsmacd.min().min():.2f} ~ {rsmacd.max().max():.2f}")

## RSMACD 信號計算

In [ ]:
# ===== RSMACD 信號計算 =====

# 前一日和前兩日的 RSMACD（用於判斷轉折）
rsmacd_prev1 = rsmacd.shift(1)  # 昨日
rsmacd_prev2 = rsmacd.shift(2)  # 前日

# 曲線轉折判斷
rsmacd_turning_up = (rsmacd > rsmacd_prev1) & (rsmacd_prev1 <= rsmacd_prev2)
rsmacd_turning_down = (rsmacd < rsmacd_prev1) & (rsmacd_prev1 >= rsmacd_prev2)


# ----- 買入信號 -----

# 綠色箭頭：RSMACD 曲線轉折向上
signal_green_arrow = rsmacd_turning_up

# 綠色三角形：RSMACD 從 30 以下突破 30（主要選股信號）
signal_green_triangle = (rsmacd > 30) & (rsmacd_prev1 <= 30)

# 綠色小球：RSMACD 轉折向上且突破 50
signal_green_ball = (rsmacd > 50) & (rsmacd_prev1 <= 50) & rsmacd_turning_up


# ----- 賣出信號 -----

# 紅色箭頭：RSMACD 曲線轉折向下
signal_red_arrow = rsmacd_turning_down

# 紅色倒三角形：RSMACD 從 70 以上跌破 70
signal_red_inv_triangle = (rsmacd < 70) & (rsmacd_prev1 >= 70)

# 紅色小球：RSMACD 轉折向下且跌破 50
signal_red_ball = (rsmacd < 50) & (rsmacd_prev1 >= 50) & rsmacd_turning_down


print("RSMACD 信號計算完成")
print(f"最新交易日: {rsmacd.index[-1].strftime('%Y-%m-%d')}")
print(f"綠色三角形（突破30）信號數: {signal_green_triangle.tail(1).sum(axis=1).iloc[0]}")
print(f"綠色箭頭（轉折向上）信號數: {signal_green_arrow.tail(1).sum(axis=1).iloc[0]}")
print(f"綠色小球（突破50）信號數: {signal_green_ball.tail(1).sum(axis=1).iloc[0]}")
print(f"紅色箭頭（轉折向下）信號數: {signal_red_arrow.tail(1).sum(axis=1).iloc[0]}")
print(f"紅色倒三角形（跌破70）信號數: {signal_red_inv_triangle.tail(1).sum(axis=1).iloc[0]}")
print(f"紅色小球（跌破50）信號數: {signal_red_ball.tail(1).sum(axis=1).iloc[0]}")

## RSMACD 綠色三角形選股篩選

In [ ]:
# ===== 分析開始時間紀錄 =====
start_time = datetime.now()
print(f"ANA001_台股選股 「RSMACD選股」 分析開始時間: {start_time}")

# ===== 篩選最新交易日出現綠色三角形的股票 =====
latest_signal = signal_green_triangle.tail(1)
latest_date = latest_signal.index[0]

# 篩選出有信號的股票代號
filtered_symbols = latest_signal.columns[latest_signal.iloc[0]].tolist()

print(f"最新交易日: {latest_date.strftime('%Y-%m-%d')}")
print(f"綠色三角形信號股票數（篩選前）: {len(filtered_symbols)}")

# ===== 5日平均成交量篩選 > 1000張 =====
vol_sma5 = vol.average(5)
vol_filter = (vol_sma5 / 1000) > 1000  # 成交股數轉張數，篩選5日均量 > 1000張
latest_vol_filter = vol_filter.tail(1)
vol_qualified_symbols = latest_vol_filter.columns[latest_vol_filter.iloc[0]].tolist()

# 取交集：同時符合信號 AND 成交量條件的股票
qualified_symbols = list(set(filtered_symbols) & set(vol_qualified_symbols))

print(f"成交量篩選後股票數: {len(qualified_symbols)}")

In [ ]:
# ===== 建立篩選結果 DataFrame =====
df_filtered = pd.DataFrame({'symbol': qualified_symbols})

# --- 股票名稱 ---
stock_name = stock_info[['stock_id', '公司簡稱']].copy()
stock_name = stock_name.rename(columns={'stock_id': 'symbol'})

# --- 最新收盤價 ---
last_close = close.tail(1).T.reset_index()
last_close.columns = ['symbol', '收盤價']

# --- 當日成交量（張）---
last_vol = vol.tail(1).T.reset_index()
last_vol.columns = ['symbol', 'vol_raw']
last_vol['vol_raw'] = last_vol['vol_raw'].fillna(0)
last_vol['成交量(張)'] = (last_vol['vol_raw'] / 1000).round().astype(int)

# --- 5日平均成交量（張）---
last_vol_sma5 = vol_sma5.tail(1).T.reset_index()
last_vol_sma5.columns = ['symbol', 'vol_sma5_raw']
last_vol_sma5['vol_sma5_raw'] = last_vol_sma5['vol_sma5_raw'].fillna(0)
last_vol_sma5['五日均量(張)'] = (last_vol_sma5['vol_sma5_raw'] / 1000).round().astype(int)

# --- RSMACD 值 ---
last_rsmacd = rsmacd.tail(1).T.reset_index()
last_rsmacd.columns = ['symbol', 'RSMACD值']
last_rsmacd['RSMACD值'] = last_rsmacd['RSMACD值'].round(2)

# --- 本益比 ---
last_pe = pe_ratio.tail(1).T.reset_index()
last_pe.columns = ['symbol', '本益比']

# --- 股價淨值比 ---
last_pb = pb_ratio.tail(1).T.reset_index()
last_pb.columns = ['symbol', '股價淨值比']

# --- 信號類型判定 ---
latest_green_triangle = signal_green_triangle.tail(1)
latest_green_arrow = signal_green_arrow.tail(1)
latest_green_ball = signal_green_ball.tail(1)

def get_signal_type(symbol):
    """判斷股票的 RSMACD 信號類型"""
    signals = []
    if symbol in latest_green_triangle.columns and latest_green_triangle[symbol].iloc[0]:
        signals.append('綠色三角形')
    if symbol in latest_green_arrow.columns and latest_green_arrow[symbol].iloc[0]:
        signals.append('綠色箭頭')
    if symbol in latest_green_ball.columns and latest_green_ball[symbol].iloc[0]:
        signals.append('綠色小球')
    return ', '.join(signals)

df_filtered['信號類型'] = df_filtered['symbol'].apply(get_signal_type)

# --- 關鍵K線高低點（確認法用）---
last_high = high.tail(1).T.reset_index()
last_high.columns = ['symbol', '關鍵K線高點']
last_low = low.tail(1).T.reset_index()
last_low.columns = ['symbol', '關鍵K線低點']

# ===== 合併所有欄位 =====
merged_df = df_filtered.merge(stock_name, on='symbol', how='left')
merged_df = merged_df.merge(last_close, on='symbol', how='left')
merged_df = merged_df.merge(last_vol, on='symbol', how='left')
merged_df = merged_df.merge(last_vol_sma5, on='symbol', how='left')
merged_df = merged_df.merge(last_rsmacd, on='symbol', how='left')
merged_df = merged_df.merge(last_high, on='symbol', how='left')
merged_df = merged_df.merge(last_low, on='symbol', how='left')
merged_df = merged_df.merge(last_pe, on='symbol', how='left')
merged_df = merged_df.merge(last_pb, on='symbol', how='left')

# ===== 排序：RSMACD值 由小到大（越低 = 超賣反轉力道越強）=====
merged_df = merged_df.sort_values('RSMACD值', ascending=True).reset_index(drop=True)

# ===== 加入看盤連結和題材概念股 =====
merged_df['web_link'] = merged_df['symbol'].apply(
    lambda x: f"https://www.wantgoo.com/stock/{x}/technical-chart"
)
merged_df['題材概念股'] = merged_df['symbol'].apply(lambda x: tst.read_topic_stocks(x))

# ===== 清理臨時欄位，重新排列輸出欄位 =====
output_columns = [
    'symbol', '公司簡稱', '收盤價', '成交量(張)', '五日均量(張)',
    'RSMACD值', '關鍵K線高點', '關鍵K線低點',
    '信號類型', '本益比', '股價淨值比',
    'web_link', '題材概念股'
]
merged_df = merged_df[[col for col in output_columns if col in merged_df.columns]]

# 重命名欄位為最終輸出格式
merged_df = merged_df.rename(columns={'symbol': '股票代號', '公司簡稱': '股票名稱'})

print(f"最終篩選結果: {len(merged_df)} 檔股票")
merged_df

In [ ]:
# 輸出報表留存
today = datetime.now().strftime("%Y%m%d")
merged_df.to_excel(f'{REPORT_PATH}/選股07_{today}.xlsx', index=False)
print(f"報表已儲存至: {REPORT_PATH}/選股07_{today}.xlsx")

## 輸出結果至Google Sheet

In [ ]:
# Google SpreadSheet 公用程式初始化
gspread_mgr = gsm.GspreadManager()
gspread_wb = gspread_mgr.get_spreadsheet(GSPERAD_SHEET_KEY)

print(f"更新Google 表單：{gspread_wb.title}，工作表：{OUTPUT_GSHEET_NAME}")

In [ ]:
# 刪除再重建工作表
gspread_mgr.recreate_worksheet(GSPERAD_SHEET_KEY, OUTPUT_GSHEET_NAME)

In [ ]:
# 更新工作表資料
# 將NaN值轉換為空字串，避免上傳時出錯
output_df = merged_df.fillna('')

gspread_mgr.update_worksheet_values(
    GSPERAD_SHEET_KEY,
    OUTPUT_GSHEET_NAME,
    [output_df.columns.values.tolist()] + output_df.values.tolist()
)

print(f"已成功更新 {len(output_df)} 筆資料至 Google SpreadSheet")

In [ ]:
# 計算總執行時間
end_time = datetime.now()
total_time = end_time - start_time

print(f"分析結束時間: {end_time}")
print(f"總執行時間: {total_time}")

---
## 信號驗證區塊 - K線圖與RSMACD信號標示

以下區塊用於驗證 RSMACD 信號的正確性，透過 K 線圖與 RSMACD 子圖視覺化呈現買賣信號。

In [ ]:
# ===== 驗證用股票代碼設定 =====
VERIFY_SYMBOL = '2330'

print(f"驗證股票代碼：{VERIFY_SYMBOL}")

In [ ]:
# ===== 取指定股票代碼 - 近2年信號輸出 =====

# 計算日期範圍（近2年）
end_date = close.index[-1]
start_date = end_date - pd.DateOffset(years=2)

# 篩選日期範圍
mask = (close.index >= start_date) & (close.index <= end_date)

# 建立驗證表格
result_df = pd.DataFrame({
    '日期': close.index[mask],
    '收盤價': close[VERIFY_SYMBOL][mask].values,
    '最高價': high[VERIFY_SYMBOL][mask].values,
    '最低價': low[VERIFY_SYMBOL][mask].values,
    'RSMACD': rsmacd[VERIFY_SYMBOL][mask].round(2).values,
    '綠色箭頭': signal_green_arrow[VERIFY_SYMBOL][mask].values,
    '綠色三角形': signal_green_triangle[VERIFY_SYMBOL][mask].values,
    '綠色小球': signal_green_ball[VERIFY_SYMBOL][mask].values,
    '紅色箭頭': signal_red_arrow[VERIFY_SYMBOL][mask].values,
    '紅色倒三角形': signal_red_inv_triangle[VERIFY_SYMBOL][mask].values,
    '紅色小球': signal_red_ball[VERIFY_SYMBOL][mask].values,
})

# 篩選有信號的日期
signal_df = result_df[
    result_df['綠色箭頭'] |
    result_df['綠色三角形'] |
    result_df['綠色小球'] |
    result_df['紅色箭頭'] |
    result_df['紅色倒三角形'] |
    result_df['紅色小球']
].copy()

# 新增信號類型欄位
def get_signal_type_for_verify(row):
    signals = []
    if row['綠色三角形']:
        signals.append('綠色三角形')
    if row['綠色箭頭']:
        signals.append('綠色箭頭')
    if row['綠色小球']:
        signals.append('綠色小球')
    if row['紅色倒三角形']:
        signals.append('紅色倒三角形')
    if row['紅色箭頭']:
        signals.append('紅色箭頭')
    if row['紅色小球']:
        signals.append('紅色小球')
    return ', '.join(signals)

signal_df['信號類型'] = signal_df.apply(get_signal_type_for_verify, axis=1)

# ===== 連續相同信號只取第一個 =====
signal_df = signal_df.reset_index()
signal_df.rename(columns={'index': '原始索引'}, inplace=True)

signal_df['前一索引'] = signal_df['原始索引'].shift(1)
signal_df['前一信號'] = signal_df['信號類型'].shift(1)
signal_df['是連續交易日'] = (signal_df['原始索引'] - signal_df['前一索引']) == 1
signal_df['信號類型相同'] = signal_df['信號類型'] == signal_df['前一信號']
signal_df['是首次信號'] = ~(signal_df['是連續交易日'] & signal_df['信號類型相同'])

first_signal_df = signal_df[signal_df['是首次信號']].copy()
first_signal_df = first_signal_df.drop(columns=['原始索引', '前一索引', '前一信號', '是連續交易日', '信號類型相同', '是首次信號'])

# ===== 加入確認法資訊 =====
def apply_confirmation(row, close_series, high_series, low_series, lookback=5):
    """套用確認法：檢查後續K線是否突破關鍵K線高點或跌破低點"""
    signal_date = row['日期']
    key_high = row['最高價']
    key_low = row['最低價']
    
    # 取信號日之後的交易日
    future_mask = close_series.index > signal_date
    future_close = close_series[future_mask].head(lookback)
    
    if len(future_close) == 0:
        return pd.Series({'確認狀態': '等待確認', '確認日期': None})
    
    for date_idx, close_val in future_close.items():
        if close_val > key_high:
            return pd.Series({'確認狀態': '已確認進場', '確認日期': date_idx.strftime('%Y-%m-%d')})
        if close_val < key_low:
            return pd.Series({'確認狀態': '已確認失敗', '確認日期': date_idx.strftime('%Y-%m-%d')})
    
    return pd.Series({'確認狀態': '等待確認', '確認日期': None})

# 只對買入信號套用確認法
buy_signal_mask = (
    first_signal_df['信號類型'].str.contains('綠色三角形') |
    first_signal_df['信號類型'].str.contains('綠色小球')
)

close_series = close[VERIFY_SYMBOL]
high_series = high[VERIFY_SYMBOL]
low_series = low[VERIFY_SYMBOL]

first_signal_df[['確認狀態', '確認日期']] = ''
if buy_signal_mask.any():
    confirmation_results = first_signal_df[buy_signal_mask].apply(
        lambda row: apply_confirmation(row, close_series, high_series, low_series),
        axis=1
    )
    first_signal_df.loc[buy_signal_mask, ['確認狀態', '確認日期']] = confirmation_results.values

# 輸出統計資訊
print(f"===== {VERIFY_SYMBOL} RSMACD 交易信號統計（{start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}）=====")
print(f"總交易日數: {len(result_df)}")
print(f"原始信號總數: {len(signal_df)}")
print(f"去除連續重複後信號數: {len(first_signal_df)}")
print()
print("各類信號次數（去除連續重複後）:")
for sig_type in ['綠色三角形', '綠色箭頭', '綠色小球', '紅色倒三角形', '紅色箭頭', '紅色小球']:
    count = first_signal_df['信號類型'].str.contains(sig_type).sum()
    print(f"  {sig_type}: {count}")
print()

# 輸出信號表格
verify_output_columns = ['日期', '收盤價', '最高價', '最低價', 'RSMACD', '信號類型', '確認狀態', '確認日期']
first_signal_df[verify_output_columns]

In [ ]:
# ===== 建立 K 線圖資料 =====

# 建立 OHLC DataFrame（包含成交量）
ohlc_df = pd.DataFrame({
    'time': close.index[mask],
    'open': open_price[VERIFY_SYMBOL][mask].values,
    'high': high[VERIFY_SYMBOL][mask].values,
    'low': low[VERIFY_SYMBOL][mask].values,
    'close': close[VERIFY_SYMBOL][mask].values,
    'volume': vol[VERIFY_SYMBOL][mask].values
})

# 計算均線
ohlc_df['MA5'] = ohlc_df['close'].rolling(window=5).mean()
ohlc_df['MA25'] = ohlc_df['close'].rolling(window=25).mean()

# 建立 RSMACD 子圖資料
rsmacd_data = pd.DataFrame({
    'time': close.index[mask],
    'RSMACD': rsmacd[VERIFY_SYMBOL][mask].values
}).dropna()

print(f"===== {VERIFY_SYMBOL} OHLC 資料（近2年）=====")
print(f"資料期間: {ohlc_df['time'].min().strftime('%Y-%m-%d')} ~ {ohlc_df['time'].max().strftime('%Y-%m-%d')}")
print(f"資料筆數: {len(ohlc_df)}")

In [ ]:
# ===== 繪製 K 線圖（上圖）與 RSMACD 指標（下圖）=====

# 建立主圖表（價格圖）
chart = JupyterChart(height=800, width=1200, inner_width=1, inner_height=0.7)

# 設定紅漲綠跌
chart.candle_style(
    up_color='#982e2e',      # 漲 - 紅色
    down_color='#205f0b',    # 跌 - 綠色
)

# 設定成交量柱狀圖顏色
chart.volume_config(
    up_color='#982e2e',
    down_color='#205f0b',
)

# 設定 OHLC 資料
chart.set(ohlc_df)

# 新增 5MA 均線 (藍色)
ma5_line = chart.create_line(name='MA5', color='#2196F3', width=2, price_label=False)
ma5_data = ohlc_df[['time', 'MA5']].dropna()
ma5_line.set(ma5_data)

# 新增 25MA 均線 (橘色)
ma25_line = chart.create_line(name='MA25', color='#FF9800', width=2, price_label=False)
ma25_data = ohlc_df[['time', 'MA25']].dropna()
ma25_line.set(ma25_data)

# ===== 建立 RSMACD 子圖 =====
subchart = chart.create_subchart(
    position='left',
    width=1,
    height=0.3,
    sync=True
)

# 繪製 RSMACD 曲線
rsmacd_line = subchart.create_line(name='RSMACD', color='#E040FB', width=2)
rsmacd_line.set(rsmacd_data)

# 繪製 30/50/70 水平參考線
subchart.horizontal_line(30, color='rgba(76, 175, 80, 0.5)', width=1, style='dashed', text='30 超賣')
subchart.horizontal_line(50, color='rgba(158, 158, 158, 0.5)', width=1, style='dashed', text='50 中性')
subchart.horizontal_line(70, color='rgba(244, 67, 54, 0.5)', width=1, style='dashed', text='70 超買')

# 自訂時間格式為 YYYY/MM/DD
chart.run_script(f'''
    {chart.id}.chart.applyOptions({{
        localization: {{
            timeFormatter: (time) => {{
                const date = new Date(time * 1000);
                const year = date.getFullYear();
                const month = String(date.getMonth() + 1).padStart(2, '0');
                const day = String(date.getDate()).padStart(2, '0');
                return year + '/' + month + '/' + day;
            }}
        }},
        timeScale: {{
            tickMarkFormatter: (time) => {{
                const date = new Date(time * 1000);
                const year = date.getFullYear();
                const month = String(date.getMonth() + 1).padStart(2, '0');
                const day = String(date.getDate()).padStart(2, '0');
                return year + '/' + month + '/' + day;
            }}
        }}
    }});
''')

# 啟用 legend
chart.legend(
    visible=True,
    ohlc=True,
    percent=True,
    color='rgb(191, 195, 203)',
    font_size=12,
    color_based_on_candle=True
)

# ===== 信號標記設定 =====
signal_config = {
    '綠色箭頭': {'position': 'below', 'shape': 'arrow_up', 'color': '#4CAF50', 'text': 'RSMACD↑'},
    '綠色三角形': {'position': 'below', 'shape': 'square', 'color': '#00E676', 'text': '突破30'},
    '綠色小球': {'position': 'below', 'shape': 'circle', 'color': '#69F0AE', 'text': '突破50'},
    '紅色箭頭': {'position': 'above', 'shape': 'arrow_down', 'color': '#F44336', 'text': 'RSMACD↓'},
    '紅色倒三角形': {'position': 'above', 'shape': 'square', 'color': '#FF5252', 'text': '跌破70'},
    '紅色小球': {'position': 'above', 'shape': 'circle', 'color': '#EF9A9A', 'text': '跌破50'},
}

# 為減少圖表雜訊，預設只顯示關鍵信號（三角形、小球）
# 設為 True 可顯示所有信號（含箭頭）
SHOW_ALL_SIGNALS = False

skip_signals = set() if SHOW_ALL_SIGNALS else {'綠色箭頭', '紅色箭頭'}

# 遍歷信號並標示在上下兩張圖
marker_count = 0
for _, row in first_signal_df.iterrows():
    signal_type = row['信號類型']
    signal_date = row['日期']
    
    for sig_type, config in signal_config.items():
        if sig_type in signal_type and sig_type not in skip_signals:
            # 標示在主圖（K線圖）
            chart.marker(
                time=pd.Timestamp(signal_date),
                position=config['position'],
                shape=config['shape'],
                color=config['color'],
                text=config['text']
            )
            # 標示在子圖（RSMACD）
            rsmacd_line.marker(
                time=pd.Timestamp(signal_date),
                position=config['position'],
                shape=config['shape'],
                color=config['color'],
                text=config['text']
            )
            marker_count += 1

print(f"===== {VERIFY_SYMBOL} K線圖 + RSMACD 信號標示 =====")
print(f"顯示模式: {'全部信號' if SHOW_ALL_SIGNALS else '關鍵信號（三角形、小球）'}")
print(f"已標示 {marker_count} 個信號")

chart.load()